### Autoencoders 
Autoencoders are a data-driven network-trained dimentionality reduction technique.  The network encodes the input image and then decodes using random weights (at initialization). The decoded image is compared with the input image to adjust the weights of the encoder-decoder. 

In principle, the encoder-decoder architecture reduces the dimentionality of the input data and tries to maintain only the most relevant information. This information is available in the bottleneck layer of the encoder-decoder architecture which is also called latent-space representation. 

An image of the top-level architecture is shown below: 


![picture](https://d3i71xaburhd42.cloudfront.net/a81895c377ba4789997a1ac3a870c0098ba5a2d5/1-Figure1-1.png)


### Some Applications: 

The following are some of the applications of Autoencoders: 

1) *Dimentionality Reduction* - its easier to understand the relationship between two variables instead of 1000 variables on each other. ☹

2) *Anomaly Detection* - the bottleneck layer provides helpful information about the relationship. So for example if some images have noise in it and most of the data is noise-free. Then the encoder will not encode *this noise* into the latent space representation (**hopefully**) and we can compare the original and recontstucted images and substact the noise. 

3) *Information Retrieval*

4) *Image Segmantation* 


In [110]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [111]:
# imports
from typing import Type
import os
import json
from datetime import datetime
import numpy as np
import tensorflow as tf
from skimage.io import imread
from skimage.transform import resize
import imageio


In [112]:
# const
MODEL_DIR = "/content/drive/MyDrive/Colab Notebooks/Week7/autoencoders/model/"
MODEL_NAME = "rtp-rgbd"
IMAGE_DIR = "/content/drive/MyDrive/Colab Notebooks/Week7/data/rtp-rgb/color_img"
EXP_DIR = "/content/drive/MyDrive/Colab Notebooks/Week7/autoencoders"
IMAGE_RESHAPE =  (256, 256, 3)
ID_TEST = ["001", "002", "003", "004", "005", "006", "007", "008", "031"]


In [113]:
def load_image_batch(img_dir, id_list, img_ext = ".png", resize_shape = None) -> np.ndarray:
    """Load a batch of images from a directory.

    Args:
        img_dir (str): Path of the directory.
        id_list (list[str]): List of image ids.
        img_ext (str, optional): Extension of the image files. Defaults to ".png".
        resize_shape (tuple[int], optional): If specified, reshape all images to this shape. Defaults to None.

    Returns:
        np.ndarray: The image data with shape (samples, width, height, channels).
    """

    image_data = []
    for id in id_list:
        img = load_image(os.path.join(img_dir, id + img_ext), resize_shape)
        image_data.append(img)

    return np.array(image_data)

def load_image(img_path, resize_shape = None) -> np.ndarray:
    """Load an image from file.

    Args:
        img_path (str): Path of the image.
        resize_shape (tuple[int], optional): If specified, reshape all images to this shape. Defaults to None.

    Returns:
        np.ndarray: The image data.
    """

    img = imread(img_path, pilmode='RGB')
    if resize_shape:
        img = resize(img, resize_shape)
    return img


In [114]:
def evaluate_batch(encoder, autoencoder, X_test, name):
  """Evaluate a batch of data.

  Args:
      model (tf.keras.Model): The model to evaluate.
      X_test (np.ndarray): Input data.
      y_test (np.ndarray): Output data.
      name (str): Name of the run.

  Returns:
      dict[str, float]: Dictionary containing metric names and values.
  """

  # Create necessary folders.
  BOTTLENECK_DIR = os.path.join(EXP_DIR, "output", name, 'bottleneck')
  RECONSTRUCTED_DIR = os.path.join(EXP_DIR, "output", name, 'reconstructed')
  ensure_dirs([BOTTLENECK_DIR, RECONSTRUCTED_DIR])

  # Evaluation.
  reconstructed_images = autoencoder.predict(X_test)
  bottleneck_images = encoder.predict(X_test)

  for i, (img_orig, img_bott, img_rec) in enumerate(zip(X_test, bottleneck_images, reconstructed_images)):
      imageio.imwrite(os.path.join(BOTTLENECK_DIR, f"encoded_{i+1}.jpg"), (img_bott * 255).astype(np.uint8))
      img_comparison = np.hstack((img_orig, img_rec))
      imageio.imwrite(os.path.join(RECONSTRUCTED_DIR, f"reconstructed_{i+1}.jpg"), (img_comparison * 255).astype(np.uint8))

  # No metrics.
  return {}
  
def ensure_dirs(dir_paths):
  """Create these directories if they do not exist yet.

  Args:
      dir_paths (list[str]): List of directory paths to create.
  """
  for dir_path in dir_paths:
      os.makedirs(dir_path, exist_ok=True)


In [115]:
def evaluate():
  # Load model.
  autoencoder = tf.keras.models.load_model(os.path.join(MODEL_DIR, MODEL_NAME))
  encoder = tf.keras.Model(autoencoder.get_layer("input_image").output, autoencoder.get_layer("bottleneck").output)

  # Load data.
  input_img = load_image_batch(IMAGE_DIR, ID_TEST, resize_shape=IMAGE_RESHAPE)

  # evaluate the model 
  evaluate_batch(encoder, autoencoder, input_img, MODEL_NAME)


In [116]:
if __name__ == "__main__":
  evaluate()